In [1]:
# data science imports
import math
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
# visualization imports
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

%matplotlib inline

In [2]:
df = pd.read_csv("Data/df_created_.csv")

In [3]:
 df.head()

,Min_IT_Loan_ID__c,Opp_Number__c,Id,AccountID,Number_Of_Loans_Granted__c,Num_Of_Loans_Paid__c,Purpose_of_Loan__c,Total_Repayments__c,Amount,Term_in_Weeks__c,Payment_Frequency__c,StageName,Applicant Age,userId,loanId,count,LoanIdFormat
0,1014484.0,6327397.0,0062x00000Ec3OhAAJ,0012x00000hYeJ7AAK,6.0,3.0,Living Expenses,4.0,1250.0,16.000,Monthly,Loan Paid,39.0,10132,12931,0,1
1,945258.0,5519027.0,0062x00000E3omCAAR,0010K00001cQWDjQAO,47.0,46.0,Travel Expenses,5.0,2050.0,20.143,Monthly,Loan Paid,43.0,37,10736,5,1
2,919931.0,5180314.0,0062x00000DrdLCAAZ,0010K00001cQWDjQAO,47.0,45.0,Travel Expenses,4.0,2050.0,16.429,Monthly,Loan Paid,43.0,37,2729,5,1
3,939812.0,5431995.0,0062x00000DVGTYAA5,0010K00001hTzwHQAS,46.0,44.0,Living Expenses,5.0,2050.0,19.857,Monthly,Loan Paid,36.0,146,2281,5,1
4,911555.0,5098387.0,0062x00000DZWmGAAX,0010K00001cQWDjQAO,47.0,44.0,Home Maintenance & Repairs,2.0,2000.0,5.000,Monthly,Loan Paid,43.0,37,2608,5,1


In [4]:
 df.columns

Index(['Min_IT_Loan_ID__c', 'Opp_Number__c', 'Id', 'AccountID',
       'Number_Of_Loans_Granted__c', 'Num_Of_Loans_Paid__c',
       'Purpose_of_Loan__c', 'Total_Repayments__c', 'Amount',
       'Term_in_Weeks__c', 'Payment_Frequency__c', 'StageName',
       'Applicant Age', 'userId', 'loanId', 'count', 'LoanIdFormat'],
      dtype='object')

In [5]:
dfTrim = df.drop(["Min_IT_Loan_ID__c", "Opp_Number__c", "Id", "AccountID",'Number_Of_Loans_Granted__c', 'Num_Of_Loans_Paid__c',
       'Purpose_of_Loan__c', 'Total_Repayments__c', 'Amount',
       'Term_in_Weeks__c', 'Payment_Frequency__c', 'StageName',
       'Applicant Age'],axis=1)

In [7]:
dfTrim.head()

,userId,loanId,count,LoanIdFormat
0,10132,12931,0,1
1,37,10736,5,1
2,37,2729,5,1
3,146,2281,5,1
4,37,2608,5,1


In [9]:
num_users = len(dfTrim.userId.unique())
num_items = len(dfTrim.loanId.unique())
num_loanFormat = len(dfTrim.LoanIdFormat.unique())

print('There are {} unique users and {} unique loans and {} unique loansformatId in this data set'.format(num_users, num_items,num_loanFormat))

There are 8179 unique users and 10576 and unique loans 43 unique loansformatId in this data set


In [10]:
# get count
df_ratings_cnt_tmp = pd.DataFrame(df.groupby('count').size(), columns=['count'])
df_ratings_cnt_tmp

,count
count,
0,1657
1,10639
2,2111
3,2088
4,467
5,95
6,17
7,5


In [12]:
dfTrim.shape, df.shape

((17079, 4), (17079, 17))

In [13]:
df.columns, dfTrim.columns

(Index(['Min_IT_Loan_ID__c', 'Opp_Number__c', 'Id', 'AccountID',
        'Number_Of_Loans_Granted__c', 'Num_Of_Loans_Paid__c',
        'Purpose_of_Loan__c', 'Total_Repayments__c', 'Amount',
        'Term_in_Weeks__c', 'Payment_Frequency__c', 'StageName',
        'Applicant Age', 'userId', 'loanId', 'count', 'LoanIdFormat'],
       dtype='object'),
 Index(['userId', 'loanId', 'count', 'LoanIdFormat'], dtype='object'))

In [14]:
ddff = dfTrim.drop_duplicates(subset=['userId'], keep='first', inplace=False)

In [15]:
ddff.shape

(8179, 4)

In [16]:
# get count
df_ratings_cnt_tmp1 = pd.DataFrame(ddff.groupby('count').size(), columns=['count'])
df_ratings_cnt_tmp1

,count
count,
0,902
1,5357
2,923
3,819
4,150
5,23
6,4
7,1


In [17]:
ddff.pivot(index='loanId', columns='userId', values='count').fillna(0)

userId,1,2,3,4,5,6,7,8,9,10,...,10134,10135,10136,10137,10138,10139,10140,10141,10142,10143
loanId,,,,,,,,,,,,,,,,,,,,,
249,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13899,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
ddff.pivot(index='LoanIdFormat', columns='userId', values='count').fillna(0)

userId,1,2,3,4,5,6,7,8,9,10,...,10134,10135,10136,10137,10138,10139,10140,10141,10142,10143
LoanIdFormat,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,0.0,4.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [23]:
 

# create a Pivot Table
pivot_table0 = ddff.pivot(index='LoanIdFormat', columns='userId', values='count').fillna(0).transpose()

# export the Pivot Table to Excel
pivot_table0.to_excel('user loan format Interaction matrix.xlsx')

In [20]:
len(ddff['userId'].unique())

8179

In [25]:
# pivot and create movie-user matrix
loan_user_mat = ddff.pivot(index='LoanIdFormat', columns='userId', values='count').fillna(0)
# create mapper from movie title to index
loan_to_idx = {
    loan: i for i, loan in 
    enumerate(list(ddff.set_index('LoanIdFormat').loc[loan_user_mat.index]))
}
# transform matrix to scipy sparse matrix
loan_user_mat_sparse = csr_matrix(loan_user_mat.values)

In [35]:
list(ddff.set_index('LoanIdFormat').loc[loan_user_mat.index])

['userId', 'loanId', 'count']

In [18]:
loan_user_mat_sparse

<10576x8179 sparse matrix of type '<class 'numpy.float64'>'
	with 9426 stored elements in Compressed Sparse Row format>

In [26]:
%env JOBLIB_TEMP_FOLDER=/tmp
# define model
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5, n_jobs=-1)
# fit
model_knn.fit(loan_user_mat_sparse)

env: JOBLIB_TEMP_FOLDER=/tmp


NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1)

In [32]:
def make_recommendation(model_knn, data,mapper, fav_loan_id, n_recommendations):
    """
    return top n similar loans recommendations based on user's input loan


    Parameters
    ----------
    model_knn: sklearn model, knn model

    data: movie-user matrix

    mapper: dict, map movie title name to index of the movie in data

    fav_movie: str, name of user input movie

    n_recommendations: int, top n recommendations

    Return
    ------
    list of top n similar movie recommendations
    """
    # fit
#     model_knn.fit(data)
    idx = fav_loan_id
    # inference
    print('Recommendation system start to make inference')
    print('......\n')
    distances, indices = model_knn.kneighbors(data[idx], n_neighbors=n_recommendations+1)
    # get list of raw idx of recommendations
    raw_recommends = \
        sorted(list(zip(indices.squeeze().tolist(), distances.squeeze().tolist())), key=lambda x: x[1])[:0:-1]
    # get reverse mapper
    reverse_mapper = {v: k for k, v in mapper.items()}
    # print recommendations
    print('Recommendations for {}:'.format(fav_loan_id))
    for i, (idx, dist) in enumerate(raw_recommends):
        print('{0}: , with distance of {1}'.format(i+1, reverse_mapper[idx], dist))

In [33]:
make_recommendation(
    model_knn=model_knn,
    data=loan_user_mat_sparse,
    mapper=loan_to_idx,
    fav_loan_id=4,
    n_recommendations=5)

Recommendation system start to make inference
......

Recommendations for 4:


KeyError: 26

In [34]:
loan_to_idx

{'userId': 0, 'loanId': 1, 'count': 2}